# Flaw Engine: Failsafe Path Discovery ♟️🚀

### 🏁 Step 1: Clone
Run this to get the code.

In [ ]:
import os
URL = "https://github.com/Gideon-gg101/flaw-engine.git"
ROOT = "/content/flaw_worker"
!rm -rf {ROOT}
!git clone {URL} {ROOT}
if os.path.exists(ROOT): 
    print("\n✅ CLONE SUCCESS!")
    print("Checking disk structure:")
    !find /content/flaw_worker -maxdepth 2 -type d

### 🏁 Step 2: Build
Run this to compile the engine.

In [ ]:
import os, shutil, glob
!apt install -y cmake g++ > /dev/null

# RECURSIVE SEARCH FOR CORE
CORE_DIR = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'CMakeLists.txt' in files and 'board.cpp' in files:
        CORE_DIR = r
        break

if not CORE_DIR:
    raise Exception("❌ Could not find C++ core directory!")

print(f"Targeting: {CORE_DIR}")
BUILD_DIR = os.path.join(CORE_DIR, "build")
if os.path.exists(BUILD_DIR): shutil.rmtree(BUILD_DIR)
os.makedirs(BUILD_DIR, exist_ok=True)

%cd {BUILD_DIR}
!cmake ..
!make -j$(nproc)

# Deploy to AI folder (find it too)
AI_DIR = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'selfplay_worker.py' in files:
        AI_DIR = r
        break

libs = glob.glob("flaw_core*.so")
if libs and AI_DIR:
    lib = os.path.abspath(libs[0])
    shutil.copy2(lib, os.path.join(AI_DIR, "flaw_core.so"))
    shutil.copy2(lib, os.path.join(os.path.dirname(AI_DIR), "flaw_core.so"))
    print("\n✅ BUILD READY!")
else:
    print("\n❌ BUILD FAILED or AI folder missing!")

### 🏁 Step 3: Run
Replace YOUR_URL with your ngrok address.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"
import sys, os

# ULTIMATE PATH DISCOVERY
PROJECT_ROOT = None
for r, dirs, files in os.walk('/content/flaw_worker'):
    if 'ai' in dirs and 'core' in dirs:
        PROJECT_ROOT = r
        break

if not PROJECT_ROOT:
    print("Trying fallback discovery...")
    for r, dirs, files in os.walk('/content/flaw_worker'):
        if 'selfplay_worker.py' in files:
            PROJECT_ROOT = os.path.dirname(r)
            break

if not PROJECT_ROOT:
    print("❌ FAIL: Could not find project folders!")
    !find /content/flaw_worker
else:
    print(f"✅ Project Root: {PROJECT_ROOT}")
    if PROJECT_ROOT not in sys.path: sys.path.insert(0, PROJECT_ROOT)
    
    # Create __init__.py if missing
    init_file = os.path.join(PROJECT_ROOT, "ai", "__init__.py")
    if not os.path.exists(init_file):
        print("Creating missing __init__.py...")
        with open(init_file, 'w') as f: pass

    from ai.selfplay_worker import DistributedWorker
    print("🚀 Starting worker...")
    worker = DistributedWorker(MASTER_URL, depth=1)
    worker.run()